In [1]:
// Cell 1: Setup Dependencies

@file:DependsOn("ai.timefold.solver:timefold-solver-core:1.14.0")
@file:DependsOn("com.fasterxml.jackson.module:jackson-module-kotlin:2.17.1")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.7.3") // For any asynchronous operations if needed

import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore
import ai.timefold.solver.core.api.score.stream.Constraint
import ai.timefold.solver.core.api.score.stream.ConstraintFactory
import ai.timefold.solver.core.api.score.stream.ConstraintProvider
import ai.timefold.solver.core.api.score.stream.Joiners
import ai.timefold.solver.core.api.score.stream.ConstraintCollectors
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.solution.PlanningScore
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.lookup.PlanningId
import ai.timefold.solver.core.api.domain.variable.PlanningVariable
import ai.timefold.solver.core.api.domain.solution.ProblemFactCollectionProperty
import ai.timefold.solver.core.api.domain.solution.PlanningEntityCollectionProperty
import ai.timefold.solver.core.api.domain.valuerange.ValueRangeProvider
import ai.timefold.solver.core.api.solver.Solver
import ai.timefold.solver.core.api.solver.SolverFactory
import ai.timefold.solver.core.config.solver.SolverConfig
import java.time.LocalTime
import java.time.DayOfWeek
import java.time.Duration



In [2]:
// Cell 2: Define Domain Models

// Location Class
data class Location(
    val name: String,
    val latitude: Double,
    val longitude: Double
)

// Skill Class
data class Skill(
    val name: String
)

// Vehicle Class
data class Vehicle(
    val id: Long,
    val licensePlate: String,
    val capacity: Int = 1 // Assuming one vehicle per agent
)

// Agent Class
data class Agent(
    @PlanningId
    val id: Long,
    val name: String,
    val skills: List<Skill>,
    val homeLocation: Location,
    val vehicle: Vehicle,
    val weeklyWorkloadHours: Int = 40 // Default 40 hours
)

// Meeting Class
@PlanningEntity
data class Meeting(
    @PlanningId
    val id: Long,
    val client: String,
    val location: Location,
    val requiredSkills: List<Skill>,
    val durationHours: Int,
    val startTime: LocalTime, // Start time of the meeting
    val endTime: LocalTime,   // End time of the meeting
    @PlanningVariable(valueRangeProviderRefs = ["agentRange"])
    var assignedAgent: Agent? = null // Planning variable
) {
    // No-arg constructor required for Timefold
    constructor() : this(
        0L,
        "",
        Location("", 0.0, 0.0),
        emptyList(),
        1,
        LocalTime.MIDNIGHT,
        LocalTime.MIDNIGHT
    )
}


In [3]:
// Cell 3: Generate Fake Data

// Parameters for Data Generation
val numberOfAgents = 5
val numberOfSkills = 3
val numberOfMeetings = 20
val locationNamePrefix = "Location"

// Predefined Skills
val predefinedSkills = listOf(
    Skill("Fire Certification"),
    Skill("Bodyguard Certification"),
    Skill("First Aid Certification")
)

// Generate Vehicles
fun generateVehicles(count: Int): List<Vehicle> {
    return (1..count).map {
        Vehicle(it.toLong(), "XYZ-00$it")
    }
}

// Generate Agents
fun generateAgents(
    count: Int,
    skills: List<Skill>,
    homeLocation: Location
): List<Agent> {
    val vehicles = generateVehicles(count)
    return (1..count).map { id ->
        Agent(
            id = id.toLong(),
            name = "Agent_$id",
            skills = skills.shuffled().take((1..skills.size).random()), // Each agent has 1 to all skills
            homeLocation = homeLocation,
            vehicle = vehicles[id - 1]
        )
    }
}

// Generate Locations (All within Geneva area for realism)
fun generateLocations(count: Int): List<Location> {
    val genevaLatitude = 46.2044
    val genevaLongitude = 6.1432
    return (1..count).map { id ->
        Location(
            name = "$locationNamePrefix$id",
            latitude = genevaLatitude + (0.01 * (Math.random() - 0.5)),
            longitude = genevaLongitude + (0.01 * (Math.random() - 0.5))
        )
    }
}

// Generate Meetings
fun generateMeetings(
    count: Int,
    skills: List<Skill>,
    locations: List<Location>
): List<Meeting> {
    return (1..count).map { id ->
        val requiredSkillCount = (1..skills.size).random()
        val requiredSkills = skills.shuffled().take(requiredSkillCount)
        val location = locations.random()
        val duration = (1..3).random() // 1 to 3 hours
        val startHour = (8..20).random()
        val startTime = LocalTime.of(startHour, 0)
        val endTime = startTime.plusHours(duration.toLong())
        Meeting(
            id = id.toLong(),
            client = "Client_$id",
            location = location,
            requiredSkills = requiredSkills,
            durationHours = duration,
            startTime = startTime,
            endTime = endTime
        )
    }
}

// Generate Home Location (Central Geneva)
val homeLocation = Location("Home_Geneva", 46.2044, 6.1432)

// Generate Agents
val agents = generateAgents(numberOfAgents, predefinedSkills, homeLocation)

// Generate Locations for Meetings
val meetingLocations = generateLocations(10)

// Generate Meetings
val meetings = generateMeetings(numberOfMeetings, predefinedSkills, meetingLocations)


In [4]:
// Cell 4: Define the Planning Solution

@PlanningSolution
data class Schedule(
    @ProblemFactCollectionProperty
    @ValueRangeProvider(id = "agentRange")
    val agents: List<Agent>,

    @PlanningEntityCollectionProperty
    val meetings: List<Meeting>
) {
    @PlanningScore
    var score: HardSoftScore? = null

    // No-arg constructor required for Timefold
    constructor() : this(emptyList(), emptyList())
}


In [5]:
// Cell 5: Define Constraints (Updated)

import ai.timefold.solver.core.api.score.stream.Joiners
import ai.timefold.solver.core.api.score.stream.ConstraintCollectors
import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore

class SecurityConstraintProvider : ConstraintProvider {
    override fun defineConstraints(constraintFactory: ConstraintFactory): Array<Constraint> {
        return arrayOf(
            // Hard Constraints
            noOverlap(constraintFactory),
            skillRequirements(constraintFactory),
            workloadLimit(constraintFactory),
            // Soft Constraints
            minimizeTotalTravelTime(constraintFactory),
            preferAgentRoomStability(constraintFactory)
        )
    }

    // Ensure that an agent is not assigned to overlapping meetings
    private fun noOverlap(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEachUniquePair(Meeting::class.java,
            Joiners.equal(Meeting::assignedAgent),
            Joiners.overlapping(
                { it.startTime.toSecondOfDay() },
                { it.endTime.toSecondOfDay() }
            )
        )
            .penalize(HardSoftScore.ONE_HARD)
            .asConstraint("No overlapping meetings")
    }

    // Ensure that the assigned agent has all required skills for the meeting
    private fun skillRequirements(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEach(Meeting::class.java)
            .filter { meeting ->
                meeting.assignedAgent != null &&
                        !meeting.requiredSkills.all { skill -> meeting.assignedAgent!!.skills.contains(skill) }
            }
            .penalize(HardSoftScore.ONE_HARD)
            .asConstraint("Agent lacks required skills")
    }

    // Ensure that an agent does not exceed their weekly workload
    private fun workloadLimit(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEach(Meeting::class.java)
            .groupBy(Meeting::assignedAgent, ConstraintCollectors.sum(Meeting::durationHours))
            .filter { agent, totalHours -> agent != null && totalHours > 40 }
            .penalize(HardSoftScore.ONE_HARD, { _, totalHours -> totalHours - 40 })
            .asConstraint("Weekly workload limit")
    }

    // Soft Constraint: Minimize total travel time (simplified as distance between meetings)
    private fun minimizeTotalTravelTime(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEach(Meeting::class.java)
            .filter { it.assignedAgent != null }
            .penalize(HardSoftScore.ONE_SOFT,
                { meeting ->
                    val agentHome = meeting.assignedAgent!!.homeLocation
                    val distance = euclideanDistance(agentHome, meeting.location)
                    (distance * 10).toInt()
                }
            )
            .asConstraint("Minimize travel time")
    }

    // Soft Constraint: Prefer agents to have consistent assignments (e.g., similar locations)
    private fun preferAgentRoomStability(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEach(Meeting::class.java)
            .filter { it.assignedAgent != null }
            .reward(HardSoftScore.ONE_SOFT,
                { meeting ->
                    val agentHome = meeting.assignedAgent!!.homeLocation
                    val distance = euclideanDistance(agentHome, meeting.location)
                    (100 - distance).toInt() // Higher reward for closer meetings
                }
            )
            .asConstraint("Prefer agent room stability")
    }

    // Helper function to calculate Euclidean distance
    private fun euclideanDistance(loc1: Location, loc2: Location): Double {
        val dx = loc1.latitude - loc2.latitude
        val dy = loc1.longitude - loc2.longitude
        return Math.sqrt(dx * dx + dy * dy)
    }
}

In [6]:
// Cell 6: Solve the problem

import ai.timefold.solver.core.config.solver.SolverConfig
import ai.timefold.solver.core.api.solver.SolverFactory
import ai.timefold.solver.core.api.solver.Solver
import java.time.Duration

// Create the solver configuration
val solverConfig = SolverConfig()
    .withSolutionClass(Schedule::class.java)
    .withEntityClasses(Meeting::class.java)
    .withConstraintProviderClass(SecurityConstraintProvider::class.java)
    .withTerminationSpentLimit(Duration.ofSeconds(30)) // Solve for 30 seconds

// Create the solver factory
val solverFactory: SolverFactory<Schedule> = SolverFactory.create(solverConfig)

// Create the initial, unsolvedxs solution
val problem = Schedule(agents, meetings)

println("Solving the problem...")
val solver: Solver<Schedule> = solverFactory.buildSolver()
val solution: Schedule = solver.solve(problem)
println("Solving finished with score: ${solution.score}")

Solving the problem...
Solving finished with score: -1hard/1980soft


In [7]:
// Cell 7: Display the results

fun formatTime(time: LocalTime): String {
    return time.format(java.time.format.DateTimeFormatter.ofPattern("HH:mm"))
}

val assignedMeetings = solution.meetings.filter { it.assignedAgent != null }
val unassignedMeetings = solution.meetings.filter { it.assignedAgent == null }

println("Assigned Meetings:")
assignedMeetings.groupBy { it.assignedAgent }.forEach { (agent, meetings) ->
    println("${agent?.name}:")
    meetings.sortedBy { it.startTime }.forEach { meeting ->
        println("  - ${meeting.client} (${formatTime(meeting.startTime)} - ${formatTime(meeting.endTime)}) - Skills: ${meeting.requiredSkills.joinToString(", ") { it.name }}")
    }
    println()
}

println("Unassigned Meetings:")
unassignedMeetings.forEach { meeting ->
    println("- ${meeting.client} (${formatTime(meeting.startTime)} - ${formatTime(meeting.endTime)}) - Skills: ${meeting.requiredSkills.joinToString(", ") { it.name }}")
}

Assigned Meetings:
Agent_2:
  - Client_2 (09:00 - 12:00) - Skills: First Aid Certification
  - Client_1 (12:00 - 14:00) - Skills: First Aid Certification, Bodyguard Certification, Fire Certification
  - Client_4 (14:00 - 16:00) - Skills: Fire Certification, Bodyguard Certification, First Aid Certification
  - Client_3 (18:00 - 21:00) - Skills: Fire Certification, Bodyguard Certification, First Aid Certification

Agent_4:
  - Client_8 (09:00 - 10:00) - Skills: First Aid Certification
  - Client_16 (10:00 - 12:00) - Skills: Bodyguard Certification, First Aid Certification
  - Client_5 (12:00 - 13:00) - Skills: Bodyguard Certification, Fire Certification, First Aid Certification
  - Client_15 (14:00 - 15:00) - Skills: Fire Certification
  - Client_9 (18:00 - 20:00) - Skills: First Aid Certification, Bodyguard Certification, Fire Certification

Agent_3:
  - Client_6 (09:00 - 10:00) - Skills: First Aid Certification
  - Client_18 (13:00 - 14:00) - Skills: First Aid Certification
  - Client_

In [8]:
// Cell 8: Analyze the score

import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore
import ai.timefold.solver.core.api.solver.SolutionManager

val solutionManager = SolutionManager.create(solverFactory)
val scoreExplanation = solutionManager.explain(solution)

println("Score: ${scoreExplanation.score}")
println("\nConstraint Matches:")
scoreExplanation.constraintMatchTotalMap.forEach { (constraintId, total) ->
    println("${constraintId}: ${total.score}")
}

Score: -1hard/1980soft

Constraint Matches:
unnamed.package/Agent lacks required skills: -1hard/0soft
unnamed.package/Minimize travel time: 0hard/0soft
unnamed.package/No overlapping meetings: 0hard/0soft
unnamed.package/Prefer agent room stability: 0hard/1980soft
unnamed.package/Weekly workload limit: 0hard/0soft


In [9]:
// Cell 8: Analyze the score (simplified)

println("Score: ${solution.score}")

// If you want to print constraint matches and they're available:
if (solution.score is ai.timefold.solver.core.api.score.constraint.ConstraintMatchTotal<*>) {
    println("\nConstraint Matches:")
    (solution.score as ai.timefold.solver.core.api.score.constraint.ConstraintMatchTotal<*>).constraintMatchSet.forEach { match ->
        println("${match.constraintName}: ${match.score}")
    }
}

Line_9.jupyter.kts (6:23 - 91) Incompatible types: ConstraintMatchTotal<*> and HardSoftScore?
Line_9.jupyter.kts (9:26 - 40) 'getter for constraintName: String!' is deprecated. Deprecated in Java

In [10]:
// Cell 8: Analyze the score (robust version)

import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore
import ai.timefold.solver.core.api.solver.SolutionManager

val solutionManager = SolutionManager.create(solverFactory)
val scoreExplanation = solutionManager.explain(solution)

println("Score: ${solution.score}")

println("\nDetailed Score Explanation:")
println("Hard Score: ${(solution.score as HardSoftScore).hardScore()}")
println("Soft Score: ${(solution.score as HardSoftScore).softScore()}")

println("\nConstraint Matches:")
scoreExplanation.constraintMatchTotalMap.forEach { (constraintId, total) ->
    println("${constraintId}: ${total.score}")
}

println("\nIndictments:")
scoreExplanation.indictmentMap.forEach { (indictedObject, indictment) ->
    println("$indictedObject: ${indictment.score}")
    indictment.constraintMatchSet.forEach { match ->
        println("  - ${match.constraintId}: ${match.score}")
    }
}

Score: -1hard/1980soft

Detailed Score Explanation:
Hard Score: -1
Soft Score: 1980

Constraint Matches:
unnamed.package/Agent lacks required skills: -1hard/0soft
unnamed.package/Minimize travel time: 0hard/0soft
unnamed.package/No overlapping meetings: 0hard/0soft
unnamed.package/Prefer agent room stability: 0hard/1980soft
unnamed.package/Weekly workload limit: 0hard/0soft

Indictments:
Meeting(id=13, client=Client_13, location=Location(name=Location4, latitude=46.20401450841131, longitude=6.140004096210173), requiredSkills=[Skill(name=Fire Certification), Skill(name=First Aid Certification)], durationHours=2, startTime=08:00, endTime=10:00, assignedAgent=Agent(id=1, name=Agent_1, skills=[Skill(name=Fire Certification), Skill(name=Bodyguard Certification)], homeLocation=Location(name=Home_Geneva, latitude=46.2044, longitude=6.1432), vehicle=Vehicle(id=1, licensePlate=XYZ-001, capacity=1), weeklyWorkloadHours=40)): -1hard/99soft
  - unnamed.package/Agent lacks required skills: -1hard/0

In [11]:
// Cell 9: Basic Visualization

val allLocations = (agents.map { it.homeLocation } + meetings.map { it.location }).distinct()
val minLat = allLocations.minOf { it.latitude }
val maxLat = allLocations.maxOf { it.latitude }
val minLon = allLocations.minOf { it.longitude }
val maxLon = allLocations.maxOf { it.longitude }

fun plotPoint(x: Double, y: Double, label: String, symbol: Char): String {
    val scaledX = ((x - minLon) / (maxLon - minLon) * 50.0).toInt()
    val scaledY = ((y - minLat) / (maxLat - minLat) * 20.0).toInt()
    return "$symbol[$label]".padStart(scaledX + 1).padEnd(51) + "\n".repeat(20 - scaledY)
}

val plot = StringBuilder()
agents.forEach { agent ->
    plot.append(plotPoint(agent.homeLocation.longitude, agent.homeLocation.latitude, agent.name, 'A'))
}
assignedMeetings.forEach { meeting ->
    plot.append(plotPoint(meeting.location.longitude, meeting.location.latitude, meeting.client, 'M'))
}

println("Agent and Meeting Locations (A: Agent, M: Meeting)")
println(plot.toString())

Agent and Meeting Locations (A: Agent, M: Meeting)
          A[Agent_1]                               









          A[Agent_2]                               









          A[Agent_3]                               









          A[Agent_4]                               









          A[Agent_5]                               









                                        M[Client_1]















           M[Client_2]                             



















         M[Client_3]                               


                                   M[Client_4]              M[Client_5]                               


M[Client_6]                                        
M[Client_7]                                        













                                        M[Client_8]















M[Client_9]                                        
                              M[Client_10]         


M[Client_11]                                       





